In [1]:
import oauth2 as oauth
import urllib2 as urllib

#-----------------------------------------------------------------------
# load our API credentials 
#-----------------------------------------------------------------------
config = {}
execfile("config.py", config)

_debug = 0

oauth_token    = oauth.Token(key=config["access_token"], secret=config["access_token_secret"])
oauth_consumer = oauth.Consumer(key=config["consumer_key"], secret=config["consumer_secret"])

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

http_method = "GET"


http_handler  = urllib.HTTPHandler(debuglevel=_debug)
https_handler = urllib.HTTPSHandler(debuglevel=_debug)

'''
Construct, sign, and open a twitter request
using the credentials from config.py.
'''

def twitterreq(url, method, parameters):
    req = oauth.Request.from_consumer_and_token(oauth_consumer,
                                             token=oauth_token,
                                             http_method=http_method,
                                             http_url=url, 
                                             parameters=parameters)

    req.sign_request(signature_method_hmac_sha1, oauth_consumer, oauth_token)

    headers = req.to_header()

    if http_method == "POST":
        encoded_post_data = req.to_postdata()
    else:
        encoded_post_data = None
        url = req.to_url()

    opener = urllib.OpenerDirector()
    opener.add_handler(http_handler)
    opener.add_handler(https_handler)

    response = opener.open(url, encoded_post_data)

    return response

def fetchsamples():
    url = "https://stream.twitter.com/1/statuses/sample.json"
    parameters = []
    response = twitterreq(url, "GET", parameters)
  
    tweets = []
    count =0
 # print len(response)
    fp = open("new_tweets.txt", "wb")
    for line in response:
        if count < 500:
            tweets.append(line.strip())
            count+=1
        else:
            break
        
    for i in tweets:
        fp.write(i + "\n")
    fp.close()
    
    fp = open("new_tweets.txt", "r")
    #print fp.read()
    fp.close()
    
#  count = 0
#  for line in response:
#      count+1
#  print count
#  print line.strip()
    

if __name__ == '__main__':
    fetchsamples()
